In [4]:
import pandas as pd
import preprocessing
import numpy as np

In [5]:
#fitbit data
wear_time = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitWearTimeViaHR_merged.csv")
dailyactivity = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyActivity_merged.csv")
dailyFitbitActiveZoneMinutes = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyFitbitActiveZoneMinutes_merged.csv")
dailyHRV = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitDailyHRV_merged.csv")
fitbitBreathingRate = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitBreathingRate_merged.csv")
fitbitSkinTemperature = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitSkinTemperature_merged.csv")
sleepDay = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepDay_merged.csv")
heartrate_15min = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\heartrate_15min_merged.csv")
sleepStageLogInfo = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepStagesDay_merged.csv")

#survey data
survey_data = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\ABriefInterventionTo_DATA_2024-09-22_1540.csv")

#coach data
coach_time = pd.read_excel("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\caoching_time.xlsx")


In [6]:
merged_final = preprocessing.merge_data(dailyactivity, wear_time, dailyFitbitActiveZoneMinutes, dailyHRV, fitbitBreathingRate, fitbitSkinTemperature, sleepDay, sleepStageLogInfo, heartrate_15min)
select_survey = preprocessing.select_survey_data(survey_data)
select_survey.to_csv('select_survey.csv')

d:\EIC-Code\00-Python\2024-08-Objective-identification-on-mental-health_CNN_LSTM\preprocessing.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
d:\EIC-Code\00-Python\2024-08-Objective-identification-on-mental-health_CNN_LSTM\preprocessing.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
d:\EIC-Code\00-Python\2024-08-Objective-identification-on-mental-health_CNN_LSTM\preprocessing.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_date

In [ ]:
fitbit_data = merged_final.copy()
#replace code 98 as 107 because the code of this participant is 107 in survey
fitbit_data.loc[fitbit_data['Id'] == 98, 'Id'] = 107

used_var = ['Id','date','TotalMinutesWearTime', 'TotalSteps', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories', 'RestingHeartRate','TotalActiveZoneMinutes', 'FatBurnActiveZoneMinutes',
       'CardioActiveZoneMinutes', 'PeakActiveZoneMinutes', 'DailyRMSSD', 'DeepRMSSD',
       'AvgBreathsPerMinute', 'LightSleepAvgBreathsPerMinute','DeepSleepAvgBreathsPerMinute', 'REMSleepAvgBreathsPerMinute',
       'TotalMinutesAsleep_x', 'TotalTimeInBed_x', 'TotalTimeAwake', 'TotalMinutesLight', 
       'TotalMinutesDeep','TotalMinutesREM', 'NightlyRelative', 'Value'] #delete 'Above', 'Below', Custom Zone; NightlyRelative represents ski temparature

used_fitbit_data = fitbit_data[used_var]

used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep','TotalTimeInBed_x':'TotalMinutesinbed',
                                   'TotalTimeAwake': 'TotalMinutesAwake',
                                   'Value': 'HR',
                                   'NightlyRelative': 'skin_temparature'}, 
                                   inplace = True)

#select participants matched with survey
all_survey_id = select_survey['Id'].unique()
used_fitbit_data = used_fitbit_data[used_fitbit_data['Id'].isin(all_survey_id)]
filtered_data = used_fitbit_data.groupby('Id').filter(lambda x: len(x) >= 10) #select dat which data points is larger than 10

In [ ]:
# impute time sequence data
miss_imputation = preprocessing.impute_missing(filtered_data)

#keep only one date
impute_data_frame = miss_imputation.copy()
impute_data_frame = impute_data_frame.reset_index().drop(columns = 'index')
date = impute_data_frame.iloc[:, 0]
impute_data_frame = impute_data_frame.drop(columns=['date'])
impute_data_frame['date'] = date

#Expand the time range in order to make the data to be time-series at the daily level because some of them missing some days data
impute_data_frame.insert(loc = 2, column = 'survey_date', value = np.nan) #add another column Fitbit data
new_data_frame = preprocessing.extend_time(impute_data_frame, select_survey)

#recode survey data
new_used_fitbit_data = preprocessing.recode_survey_time(new_data_frame, select_survey) #recode survey time, complete first follow-up, complete second follow-up

#add coach time
fitbit_coach = preprocessing.recode_coach_date(new_used_fitbit_data, coach_time) #recode coach date and combine with Fitbit

# impute extend data
miss_imputation2 = preprocessing.impute_missing(fitbit_coach)
#keep only one date
impute_data_frame2 = miss_imputation2.copy()
impute_data_frame2 = impute_data_frame2.reset_index().drop(columns = 'index')
date = impute_data_frame2.iloc[:, 0]
impute_data_frame2 = impute_data_frame2.drop(columns=['date'])
impute_data_frame2['date'] = date

impute_data_frame2.to_csv('impute_data_frame2.csv')


In [ ]:
select_survey['weeks_followup_survey_96ac_complete'].value_counts() #87 sample size, We only select the participants who completed the second follow-up
#check the individual who did not complete second follow-up survey
select_survey[select_survey['weeks_followup_survey_96ac_complete'] == 0]['Id']
# 40      42
# 46      48
# 67      69
# 107    109dd
# 112    114
# 123    125
# 129    131
# 132    134